In [ ]:
# use psi4 to compute energy of diargon molecule from 3.5-6 angstrom with 0.1 angstrom step
# then use scipi curve_fit to fit curve to lennard jones potential and plot comparison of data and fit
import psi4
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# define lennard jones potential
def lj(r, eps, sig):
    return 4*eps*((sig/r)**12 - (sig/r)**6)

# define function to compute energy of diargon molecule
def compute_energy(r):
    # set molecule geometry
    mol = psi4.geometry(f"""
    0 1
    Ar
    --
    Ar 1 {r}
    symmetry c1
    """)
    # set basis set
    psi4.set_options({'basis': 'sto-3g'})
    # compute energy
    energy = psi4.energy('scf')
    return energy

# run compute_energy function for range of distances
r = np.arange(3.5, 6.1, 0.1)
energy = np.array([compute_energy(i) for i in r])

# fit curve to data
popt, pcov = curve_fit(lj, r, energy)

# plot data and fit
plt.plot(r, energy, 'o', label='data')
plt.plot(r, lj(r, *popt), label='L-J fit')
plt.xlabel('r (Angstrom)')
plt.ylabel('Energy (Hartree)')
plt.legend()
plt.show()






In [ ]:
#! Example potential energy surface scan and CP-correction for Ar2

ar2_geometry = """
0 1
Ar
--
Ar 1 {0}
symmetry c1
"""

Rvals = [3.5, 6.0, 0.1]

psi4.set_options({'freeze_core': 'true'})

# Initialize a blank dictionary of counterpoise corrected energies
# (Need this for the syntax below to work)

ecp = {}

for R in Rvals:
    ar2 = psi4.geometry(ar2_geometry.format(R))
    ecp[R] = psi4.energy('ccsd(t)/aug-cc-pvdz', bsse_type='cp', molecule=ar2)

# Prints to screen
print("CP-corrected CCSD(T)/aug-cc-pVDZ Interaction Energies\n\n")
print("          R [Ang]                 E_int [kcal/mol]       ")
print("---------------------------------------------------------")
for R in Rvals:
    e = ecp[R] * psi4.constants.hartree2kcalmol
    print("            {:3.1f}                        {:1.6f}".format(R, e))

# Prints to output.dat
psi4.core.print_out("CP-corrected CCSD(T)/aug-cc-pVDZ Interaction Energies\n\n")
psi4.core.print_out("          R [Ang]                 E_int [kcal/mol]       \n")
psi4.core.print_out("---------------------------------------------------------\n")
for R in Rvals:
    e = ecp[R] * psi4.constants.hartree2kcalmol
    psi4.core.print_out("            {:3.1f}                        {:1.6f}\n".format(R, e))


In [ ]:
# define lennard jones potential
def lj(r, eps, sig):
    return 4*eps*((sig/r)**12 - (sig/r)**6)

# convert dictionary to numpy array
energy = np.array(list(ecp.values()))

# fit curve to data
popt, pcov = curve_fit(lj, r, energy)

# plot data and fit
plt.plot(r, energy, 'o', label='data')
plt.plot(r, lj(r, *popt), label='L-J fit')
plt.xlabel('r (Angstrom)')
plt.ylabel('Energy (Hartree)')
plt.legend()
plt.show()
